# Snapista Notebook - Snow Cover with Sentinel-2

In [5]:
import os
import sys
gpt_path = '/srv/conda/envs/env_snap/snap/bin/gpt'
if not gpt_path in sys.path: sys.path.append(gpt_path)
if not os.getcwd() in sys.path: sys.path.append(os.getcwd())

from snapista import Operator, OperatorParams
from snapista import Graph
from snapista import TargetBand, TargetBandDescriptors

from stac import *
    
# gpt_path = '/srv/conda/envs/env_snap/snap/bin/gpt'
# if not gpt_path in sys.path: sys.path.append(gpt_path)

ImportError: cannot import name 'GPF' from 'snappy' (unknown location)

## Helper Functions

In [ ]:
def buildVRT(vrt, assets, resolution, nodata):
    ds = gdal.BuildVRT(vrt,
                       assets,
                       options=gdal.BuildVRTOptions(srcNodata=nodata,
                                                    resolution='highest',
#                                                     xRes=10, 
#                                                     yRes=10,
                                                    separate=True)
                      )
    ds.FlushCache() 
    ds = None; del(ds) # these are needed to generate the VRT! 

## Read Data

In [ ]:
aoi_etna = 'POLYGON ((14.9 37.653, 14.9 37.816, 15.104 37.816, 15.104 37.653, 14.9 37.653))' #'14.9,37.653,15.104,37.816'
aoi_tutorial = 'POLYGON ((12.7505 49.3057, 13.7601 49.3057, 13.76011 48.7043, 12.7505 48.7043, 12.7505 49.3057, 12.7505 49.3057))'

In [ ]:
# input_path = '/workspace/stage-in/s2_tutorial'
input_path = '/workspace/stage-in/vrss2-mss/'
# input_path = '/workspace/stage-in/s2_etna_20210402'
item = get_item(os.path.join(input_path, "catalog.json"))
item

In [ ]:
# Upload assets
# item_out = Item(id=item.id,
#                 geometry=item.geometry,
#                 bbox=item.bbox,
#                 datetime=item.datetime,
#                 properties=item.properties)

# item_out.properties['title'] += ' - SnowCover'

prefix = item.id 

output_dir = item.id 
if not os.path.exists(output_dir): os.makedirs(output_dir)

#==> Generate VRT of MS bands (without Pan nor Thermal bands)
assets_dict = {}

eo_item = extensions.eo.EOItemExt(item)
# print(eo_item.bands)

for band in eo_item.bands:
    
    assets_dict[band.common_name] = get_asset(item, band).href
    
# [print(key,assets_dict[key]) for key in assets_dict.keys()]

assets = list(assets_dict.values())
for a in assets: print(a)

In [ ]:
# Generate VRT of bands
nodata = 0
vrt = os.path.join(output_dir,f'{prefix}.vrt')
print('- vrt name: ' + vrt)
buildVRT(vrt, assets, resolution='highest', nodata=nodata)

In [ ]:
green = assets_dict['green'] # B3
red = assets_dict['red'] # B4
rededge = assets_dict['rededge70'] # B5
nir = assets_dict['nir'] # B8
swir16 = assets_dict['swir16'] # B11
swir22 = assets_dict['swir22'] # B12
slc = assets_dict['slc'] # not sure this will work, as it doesnt have common_name

## Pre-processing

## Read

In [ ]:
read = Operator('Read', file=vrt)
read.formatName = 'SENTINEL-2'
read

In [ ]:
resample = Operator('Resample')
resample

In [ ]:
# Define Cloud Mask
# SLC legend --> https://www.sentinel-hub.com/faq/how-get-s2a-scene-classification-sentinel-2/
cloudmask = TargetBand(name = 'cloudmask',
                       expression = f'({slc} == 9 ? 2 : ({slc} > 8) & ({green}>0.3) ? 1 : 0')
bandmaths_1 = Operator('BandMaths')
bandmaths_1

In [ ]:
# Define Normalised Difference Snow Index (NDSI)
ndsi = TargetBand(name = 'ndsi',
                  expression = f'({green} - {swir16}) / ({green} + {swir16})')
bandmaths_2 = Operator('BandMaths')
bandmaths_2.targetBandDescriptors = TargetBandDescriptors([ndsi])
bandmaths_2

In [ ]:
bandmerge = Operator('BandMerge')
bandmerge

In [ ]:
# Define spectral and spatial subset 
subset = Operator('Subset')
# aoi_etna or aoi_tutorial
subset

In [ ]:
write = Operator('Write', output=)
write

## Create graph

In [ ]:
g = Graph()
g

In [ ]:
g.add_node(read, 
           node_id='read')

g.add_node(resample, 
           node_id='resample', 
           source='read')

g.add_node(bandmaths_1, 
           node_id='bandmaths_1', 
           source='resample')

g.add_node(bandmaths_2, 
           node_id='bandmaths_2', 
           source='bandmaths_1')

g.add_node(bandmerge, 
           node_id='bandmerge', 
           source='bandmaths_2')

g.add_node(resample, 
           node_id='resample', 
           source='bandmerge')

g.add_node(subset, 
           node_id='subset', 
           source='resample')

Instantiate a SNAP operator by providing its name and update a parameter value

In [ ]:
calibration.createBetaBand = 'true'

calibration

Instantiate a SNAP operator by providing its name and setting the values in the constructor 

In [ ]:
calibration = Operator('Calibration', 
                       createBetaBand='true', 
                       createGammaBand='true')

calibration

In [ ]:
print(calibration)

# Graph

Create a graph:

In [ ]:
g = Graph()

g

In [ ]:
print(g)

Add two nodes:

In [ ]:
g.add_node(operator=Operator('Read'), 
           node_id='read_1')

calibration = Operator('Calibration')

calibration.createBetaBand = 'false'

g.add_node(operator=calibration, 
           node_id='calibration', 
           source='read_1')

In [ ]:
g.view()

In [ ]:
print(g)

## Create a linear graph

In [ ]:
g = Graph() 

read = Operator('Read')

read.formatName = 'SENTINEL-1'
read.file = 'some1'


operators = [read,
             'Resample',
             'Reproject',
             'Subset',
             'AddLandCover',
             'Write']

for index, operator in enumerate(operators):
    
    print('Adding Operator {} to graph'.format(operator.operator if isinstance(operator, Operator) else operator))
    if index == 0:            
        source_node_id = ''

    else:
        source_node_id = operators[index - 1].operator if isinstance(operators[index - 1], Operator) else operators[index - 1]
        
        
    g.add_node(operator if isinstance(operator, Operator) else Operator(operator),
               operator.operator if isinstance(operator, Operator) else operator,
               source_node_id)
      

In [ ]:
g.view()

In [ ]:
#g.run()

# Examples

## BandMaths

In [ ]:
band_maths = Operator('BandMaths')

In [ ]:
band_maths.describe()

<targetBand>
          <name>active_fire_detected</name>
          <type>float32</type>
          <expression>S9_BT_in &lt; 265 ? 0 : F1_BT_in &gt; 315 and (F1_BT_in - F2_BT_in) &gt; 15 ? 1 : 0</expression>
          <description/>
          <unit/>
          <noDataValue>NaN</noDataValue>
        </targetBand>

In [ ]:
snapista.

In [ ]:
active_fire_band = TargetBand(name='active_fire_detected',
                              expression='S9_BT_in &lt; 265 ? 0 : F1_BT_in &gt; 315 and (F1_BT_in - F2_BT_in) &gt; 15 ? 1 : 0')

active_fire_band

In [ ]:
band_maths.targetBandDescriptors = TargetBandDescriptors([active_fire_band])

band_maths.targetBandDescriptors

In [ ]:
g = Graph()

g.add_node(operator=Operator('Read'), 
           node_id='read_1')

In [ ]:
g.add_node(operator=band_maths, 
           node_id='band_maths', 
           source='read_1')

In [ ]:
g.view()

## Operator Subset

In [ ]:
subset = Operator('Subset')

subset.describe()

In [ ]:
subset.geoRegion = 'POLYGON((12.76 41.73,12.84 41.73,12.84 41.68,12.76 41.68,12.76 41.73))'
subset

In [ ]:
g.add_node(operator=subset, 
           node_id='subset', 
           source='the_previous_node_id')

In [ ]:
# you can also define it straight away when adding the node, ie
subset2 = Operator('Subset', geoRegion = 'POLYGON((...))')
subset2

In [ ]:
subset3 = Operator("Subset", copyMetadata = True, geoRegion='POLYGON((12.767154479927116 41.73189894469486,12.844058776802116 41.73189894469486,12.844058776802116 41.68063637079049,12.767154479927116 41.68063637079049,12.767154479927116 41.73189894469486))'),
subset3

In [ ]:
from snapista import *

In [ ]:
snapista

## Operator Write

In [ ]:
write = Operator('Write')
write.describe()

## Other

In [ ]:
import numpy as np

# List sorted operators
sorted_list = np.sort(Graph.list_operators()).tolist()
print('Length:',len(sorted_list))
sorted_list

In [ ]:
# print nicely in a 4 column list
for a,b,c,d in zip(sorted_list[:60],sorted_list[61:120],sorted_list[121:180],sorted_list[181:]): 
    print('{:<30}{:<30}{:<30}{:<30}'.format(a,b,c,d))

In [ ]:
calibration = Operator('calibration')
calibration.describe()

In [ ]:
# List of Operators with description, sorted

In [ ]:
import os
import subprocess
import tempfile
import lxml.etree as etree
from snappy import GPF

In [ ]:
# Function to get operators with description
def describe_operators_manual():
    
    desc = {}
    
    GPF.getDefaultInstance().getOperatorSpiRegistry().loadOperatorSpis()

    op_spi_it = GPF.getDefaultInstance().getOperatorSpiRegistry().getOperatorSpis().iterator()

    while op_spi_it.hasNext():

        op_spi = op_spi_it.next()
        
        alias = op_spi.getOperatorDescriptor().getAlias()
        description = op_spi.getOperatorDescriptor().getDescription()
        
        desc[alias] = description
        print(
            "{} - {}".format(
                alias, description
            )
        )
    return desc

In [ ]:
desc = describe_operators_manual()

In [ ]:
# List sorted operators

print(sorted(desc.keys()))



# np.sort(Graph.list_operators()).tolist()
print('Length:',len(desc))
# sorted_list